In [ ]:
#TB! load env ariables
import our_utilities; our_utilities.load_env_with_substitutions()

from dagster import asset, AssetIn, AssetKey, AssetMaterialization, AssetExecutionContext, Output, DynamicOutput, DynamicOut
from dagster.core.instance import DagsterInstance

# Import DataFrame from pandas
from pandas import DataFrame

import pandas as pd

from ...resources.resources import MyAWSS3Resource

@asset
def get_new_s3_files(context: AssetExecutionContext,
                     s3_with_bucket: MyAWSS3Resource):
    s3_client = s3_with_bucket.create_s3_client()
    bucket = s3_with_bucket.bucket_name

    dagster_instance = DagsterInstance.get()

    paginator = s3_client.get_paginator('list_objects')
    new_files = []
    for result in paginator.paginate(Bucket=bucket):
        for file in result.get('Contents', []):
            file_name = file['Key']
            if not dagster_instance.has_asset_key(AssetKey(file_name)):
                new_files.append({'bucket': bucket, 'key': file_name})
    return new_files


@asset(out=DynamicOut(dagster_type=DataFrame),
       ins={"new_files": AssetIn(key=AssetKey("get_new_s3_files"))})
def create_assets_from_s3_files(context: AssetExecutionContext, 
                                s3_with_bucket: MyAWSS3Resource,
                                new_files: list):
    s3_client = s3_with_bucket.create_s3_client()
    for s3_coordinate in new_files:
        bucket = s3_coordinate['bucket']
        key = s3_coordinate['key']
        data = s3_client.get_object(Bucket=bucket, Key=key)

        # first test if the data is a csv file
        if data['ContentType'] != 'text/csv':
            # continue to the next file
            continue
        
        df = pd.read_csv(data['Body'])
        yield AssetMaterialization(asset_key=key, description="Created asset from S3 file")
        yield DynamicOut(df, output_name=key)  # Yield a dynamic output for each file






In [ ]:
#this seemed to work

#TB! load env ariables
import our_utilities; our_utilities.load_env_with_substitutions()

from dagster import asset, AssetIn, AssetKey, AssetMaterialization, AssetExecutionContext, Output, DynamicOutput, DynamicOut
from dagster.core.instance import DagsterInstance

# Import DataFrame from pandas
from pandas import DataFrame

import pandas as pd

from ...resources.resources import MyAWSS3Resource

@asset
def get_new_s3_files(context: AssetExecutionContext,
                     s3_with_bucket: MyAWSS3Resource):
    s3_client = s3_with_bucket.create_s3_client()
    bucket = s3_with_bucket.bucket_name

    dagster_instance = DagsterInstance.get()

    paginator = s3_client.get_paginator('list_objects')
    new_files = []
    for result in paginator.paginate(Bucket=bucket):
        for file in result.get('Contents', []):
            file_name = file['Key']
            if not dagster_instance.has_asset_key(AssetKey(file_name)):
                new_files.append({'bucket': bucket, 'key': file_name})
    return new_files


@asset(out=DynamicOut(dagster_type=DataFrame),
       ins={"new_files": AssetIn(key=AssetKey("get_new_s3_files"))})
def create_assets_from_s3_files(context: AssetExecutionContext, 
                                s3_with_bucket: MyAWSS3Resource,
                                new_files: list):
    s3_client = s3_with_bucket.create_s3_client()
    for s3_coordinate in new_files:
        bucket = s3_coordinate['bucket']
        key = s3_coordinate['key']
        data = s3_client.get_object(Bucket=bucket, Key=key)

        # first test if the data is a csv file
        if data['ContentType'] != 'text/csv':
            # continue to the next file
            continue
        
        df = pd.read_csv(data['Body'])
        yield AssetMaterialization(asset_key=key, description="Created asset from S3 file")
        yield DynamicOut(df, output_name=key)  # Yield a dynamic output for each file


